# Setup

In [ ]:
%%bash
python ./generate_data.py
python ./run_ddl.py

In [ ]:
%%sql --show
use prod.db

In [ ]:
%%sql
SELECT
    l_orderkey,
    round( sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)),
        2
    ) AS totalprice
FROM
    lineitem
WHERE
    l_orderkey = 1
GROUP BY
    l_orderkey;


In [ ]:
%%sql

-- The totalprice of an order (with orderkey = 1)
SELECT
    o_orderkey,
    o_totalprice
FROM
    orders
WHERE
    o_orderkey = 1;



In [ ]:
%%sql
    CREATE TABLE dim_customer AS 
 SELECT 
        c.*,
        n_name AS nation_name,
        n_comment AS nation_comment,
        r_name AS region_name,
        r_comment AS region_comment
    FROM customer c
    LEFT JOIN nation n ON c_nationkey = n_nationkey
    LEFT JOIN region r ON n_regionkey = r_regionkey

In [ ]:
%%sql
CREATE TABLE fct_orders as select * from orders

In [ ]:
%%sql
CREATE TABLE fct_lineitem as select * from lineitem

In [ ]:
%%sql
create table wide_orders as 
 SELECT 
        f.*,
        d.*
    FROM fct_orders f
    LEFT JOIN dim_customer d ON f.o_custkey = d.c_custkey

In [ ]:
%%sql
create table wide_lineitem as select * from fct_lineitem

In [ ]:
%%sql
create table order_lineitem_metrics as 
SELECT 
        l_orderkey as order_key,
        COUNT(l_linenumber) AS num_lineitems
    FROM wide_lineitem
    GROUP BY l_orderkey

In [ ]:
%%sql
create table customer_outreach_metrics as 
SELECT 
        wo.c_custkey,
        wo.c_name AS customer_name,
        MIN(wo.o_totalprice) AS min_order_value,
        MAX(wo.o_totalprice) AS max_order_value,
        AVG(wo.o_totalprice) AS avg_order_value,
        AVG(olm.num_lineitems) AS avg_num_items_per_order
    FROM wide_orders wo
    LEFT JOIN order_lineitem_metrics olm ON wo.o_orderkey = olm.order_key
    GROUP BY wo.c_custkey, wo.c_name

In [ ]:
%%sql --show
select *
from customer_outreach_metrics